In [1]:
state = 'WV' 
year = 2010

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json
from epsilon_constraint import*
import networkx as nx

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of WV has 3 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from WV_plans import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 0 plans from file.


In [6]:
obj_type = 'inverse_Polsby_Popper'
starting_deviation = 0.01 
verbose = True
contiguity = 'lcut'               # {'lcut', 'scf', 'shir'}
warm_start_mode = 'user'          # {'None', 'user', 'refinement'}
warm_starts = None                # if you have user define warm strats else it is None
time_limit = 7200                 # Time limit in seconds, adjust as needed
multiplier = 0                    # Adjust multiplier as needed
max_B = True                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True
symmetry_breaking = 'orbitope'    # {None, 'orbitope', 'rsum'}              
sizes = None

In [ ]:
print(f"Running epsilon constraint method for state {state}, objective {obj_type}")
print(f"{'#' * 100}\n")
(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity,        
            cutoff=None,       
            verbose=verbose,
            warm_start_mode = warm_start_mode,
            warm_starts=warm_starts,  
            starting_deviation=starting_deviation, 
            time_limit=time_limit, 
            sizes=sizes,      
            max_B=max_B,      
            symmetry_breaking=symmetry_breaking, 
        )

Running epsilon constraint method for state WV, objective inverse_Polsby_Popper
####################################################################################################

L = 611489 and U = 623841

****************************************
Trying deviation = 6176.6466666666665
****************************************
No valid warm start used.

****************************************
Running labeling model!
****************************************
L = 611489 and U = 623841
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - W

Thread count was 20 (of 20 available processors)

Solution count 5: 4.47087 4.66768 4.88598 ... 5.70028

Optimal solution found (tolerance 0.00e+00)
Best objective 4.470874618571e+00, best bound 4.470874618571e+00, gap 0.0000%

User-callback calls 3578, time in user-callback 1.39 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
[[1, 8, 15, 18, 25, 28, 30, 38, 40, 41, 43, 44, 50, 53], [0, 2, 3, 4, 5, 6, 10, 11, 16, 17, 20, 29, 31, 32, 33, 35, 45, 48, 49, 52], [7, 9, 12, 13, 14, 19, 21, 22, 23, 24, 26, 27, 34, 36, 37, 39, 42, 46, 47, 51, 54]]

****************************************
Trying deviation = 4612.166666666705
****************************************
No valid warm start used.

****************************************
Running labeling model!
****************************************
L = 613053 and U = 622276
sizes =  [1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set par

H 2500  1130                       6.2933782    3.85420  38.8%  80.0   11s
H 4742  1641                       6.1223935    4.26603  30.3%  74.2   14s
H 4780  1323                       5.7601857    4.26603  25.9%  74.1   14s
H 4792  1134                       5.5580418    4.26603  23.2%  74.0   14s
  5367  1267    5.14638   22  248    5.55804    4.36211  21.5%  72.3   15s
H 6115  1114                       4.7901629    4.49636  6.13%  71.4   16s
* 6728   177              42       4.6075923    4.53341  1.61%  67.1   16s

Cutting planes:
  Gomory: 6
  Clique: 2
  Zero half: 17
  RLT: 2
  BQP: 1
  Lazy constraints: 79

Explored 7244 nodes (460739 simplex iterations) in 17.06 seconds (6.01 work units)
Thread count was 20 (of 20 available processors)

Solution count 9: 4.60759 4.79016 5.55804 ... 7.98557

Optimal solution found (tolerance 0.00e+00)
Best objective 4.607592275004e+00, best bound 4.607592275004e+00, gap 0.0000%

User-callback calls 16095, time in user-callback 1.33 sec

******

     0     0    2.92254    0  218          -    2.92254      -     -    1s
     0     0    2.92429    0  211          -    2.92429      -     -    1s
     0     0    2.92737    0  211          -    2.92737      -     -    1s
     0     0    2.93479    0  214          -    2.93479      -     -    2s
     0     0    2.93629    0  223          -    2.93629      -     -    2s
     0     0    2.94415    0  231          -    2.94415      -     -    2s
     0     0    2.94693    0  240          -    2.94693      -     -    2s
     0     0    2.94693    0  229          -    2.94693      -     -    2s
     0     0    2.94693    0  228          -    2.94693      -     -    2s
     0     2    2.94728    0  227          -    2.94728      -     -    2s
   781   703    7.59585   35  217          -    3.56939      -  72.3    5s
  1319  1121    5.28268   29  126          -    3.78958      -  86.6   10s
H 1650  1236                       8.0189296    3.78958  52.7%  83.6   10s
H 1662  1203             

     0     0    2.91644    0  193          -    2.91644      -     -    0s
     0     0    2.91822    0  194          -    2.91822      -     -    0s
     0     0    2.96486    0  210          -    2.96486      -     -    0s
     0     0    2.96729    0  209          -    2.96729      -     -    0s
     0     0    2.97334    0  217          -    2.97334      -     -    1s
     0     0    2.98170    0  211          -    2.98170      -     -    1s
     0     0    2.98334    0  212          -    2.98334      -     -    1s
     0     0    2.98724    0  212          -    2.98724      -     -    1s
     0     2    2.99338    0  212          -    2.99338      -     -    1s
   773   682    6.89702   42  241          -    3.64438      -  71.2    5s
  2527  1594    5.99142   30  172          -    3.66834      -  86.3   10s
  7768  5783    4.96506   41   68          -    3.66834      -  73.7   15s
 12812  9450    6.75695   30  215          -    4.30929      -  68.4   20s
H14966  6742             

Best objective 5.546070848389e+00, best bound 5.546070848389e+00, gap 0.0000%

User-callback calls 66040, time in user-callback 2.22 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
[[1, 8, 14, 15, 18, 19, 25, 28, 38, 40, 44, 53], [0, 2, 3, 4, 5, 6, 10, 11, 16, 17, 30, 31, 32, 33, 35, 41, 45, 48, 49, 50], [7, 9, 12, 13, 20, 21, 22, 23, 24, 26, 27, 29, 34, 36, 37, 39, 42, 43, 46, 47, 51, 52, 54]]

****************************************
Trying deviation = 702.1666666667055
****************************************
No valid warm start used.

****************************************
Running labeling model!
****************************************
L = 616963 and U = 618366
sizes =  [1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07


In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
# Determine the senses for the Pareto frontier
# Minimize the first objective ('deviation').
# Maximize the second objective for compactness types, otherwise minimize it.
senses = ['min', 'max' if obj_type in ['average_Polsby_Popper','bottleneck_Polsby_Popper'] else 'min']
obj_names = ['deviation_persons', obj_type]
infeasible_region = [0, min(round(r[2],1) for r in result)]

In [ ]:
from pareto import ParetoFrontier

print(f"\n{'#' * 100}\nPareto Frontier for state {state},  objective {obj_type}\n{'#' * 100}\n")
pareto = ParetoFrontier(senses, obj_names, state=state, level='county')

if not  plans:
    print("No plan found!")
else:
    for plan, obj_bound, dev in result:
        upper_bound = [dev,  1/obj_bound[0] if  obj_type == 'bottleneck_Polsby_Popper' else obj_bound[0]]
        lower_bound = [dev,  1/obj_bound[1] if  obj_type == 'bottleneck_Polsby_Popper' else obj_bound[1]]
        
        pareto.add_plan(plan, upper_bound, lower_bound)
            
print("Pareto front plans:", pareto.plans)
print("Pareto front upper bounds:", pareto.upper_bounds)
print("Pareto front lower bounds:", pareto.lower_bounds)
    
#extra_points: list of tuples, each containing (deviation, objective_value, label)
if  year == 2010:
    
    #2010 enacted map scores
    enacted_map_deviation = 3197.333333333372
    enacted_map_scores = {'inverse_Polsby_Popper': 7.75, 'cut_edges': 34.00, 
                                      'perimeter': 42.12, 'average_Polsby_Popper': 0.14,'bottleneck_Polsby_Popper': 0.10}
    # Cooper plan 1
    Cooper_plan_1_deviation = 323.66666666662786
    Cooper_plan_1_scores = {'inverse_Polsby_Popper': 7.31, 'cut_edges': 34.00, 
                                      'perimeter': 40.53, 'average_Polsby_Popper': 0.17,'bottleneck_Polsby_Popper': 0.10}
    # Cooper plan 2
    Cooper_plan_2_deviation = 232.66666666662786
    Cooper_plan_2_scores = {'inverse_Polsby_Popper': 8.18, 'cut_edges': 36.00, 
                                      'perimeter': 43.36, 'average_Polsby_Popper': 0.16,'bottleneck_Polsby_Popper': 0.09}
    # Cooper plan 3
    Cooper_plan_3_deviation = 115.66666666662786
    Cooper_plan_3_scores = {'inverse_Polsby_Popper': 7.25, 'cut_edges': 35.00, 
                                      'perimeter': 40.65, 'average_Polsby_Popper': 0.16,'bottleneck_Polsby_Popper': 0.09}
    extra_points = [
                (enacted_map_deviation,  enacted_map_scores[obj_type], 'Enacted Map'),
                (Cooper_plan_1_deviation,  Cooper_plan_1_scores[obj_type] , 'Cooper plan 1'),
                (Cooper_plan_2_deviation,  Cooper_plan_2_scores[obj_type], 'Cooper plan 2'),
                (Cooper_plan_3_deviation, Cooper_plan_3_scores[obj_type], 'Cooper plan 3'),
                        ]
    
    #extra_colors: list of colors corresponding to the points in extra_points
    extra_colors = ['r', 'g', 'c', 'y']  # Red, Green, Cyan, Yellow for each of the extra points
    
    for ep in extra_points:
        print(f"The {ep[2]} has an objective value of {ep[1]} and a deviation of {ep[0]}.")
else:
    extra_points = None
    extra_colors = None
        
pareto.tighten_lower_bounds()
    
if infeasible_region[1] > 0.5:
    max_dev = max(upper_bound[0] for upper_bound in pareto.upper_bounds)
    max_ce = max(upper_bound[1] for upper_bound in pareto.upper_bounds)
    o1lim=[-1, max_dev+100]
    o2lim=[-2, max_ce*(1.1)]
else:
    o1lim = None
    o2lim = None

pareto.plot_with_custom_x_ranges(method = 'epsilon_constraint_method', splits=None, 
                                 o1lim=o1lim, o2lim=o2lim, infeasible_region = infeasible_region,
                                 extra_points=extra_points, extra_colors=extra_colors)

In [ ]:
from draw import *

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: round(x, 0)}

for plan, obj_bound, dev in result:
    ub = format_obj.get(obj_type, lambda x: round(x, 4))(obj_bound[0])
    lb = format_obj.get(obj_type, lambda x: round(x, 4))(obj_bound[1])
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    if ub != lb:
        title = f"{round(dev, 2)}-person deviation ({deviation_percentage}%), obj_bounds [{ub}, {lb}] {obj_type}"
    else:
        title = f"{round(dev, 2)}-person deviation ({deviation_percentage}%), with {ub} {obj_type}"

    draw_plan(filepath, filename2, G, plan, title=title, year=year)